In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year = 2013)

In [4]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B01003_001E","B19013_001E", "B17001_002E", "B23025_005E"), 
                         {'for': 'zip code tabulation area:*'})

In [5]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

In [6]:
# Column Reordering
census_pd = census_pd.rename(columns={"B19013_001E": "Household Income",
                                      "B01003_001E": "Population",
                                     "B17001_002E": "Poverty Count",
                                     "B23025_005E": "Unemployment Count",
                                     "NAME": "Name", "zip code tabulation area": "Zipcode"})
census_pd.head()

,Population,Poverty Count,Household Income,Unemployment Count,Name,Zipcode
0,18450.0,10816.0,12041.0,1464.0,ZCTA5 00601,00601
1,41302.0,22409.0,15663.0,4767.0,ZCTA5 00602,00602
2,53683.0,26220.0,15485.0,4341.0,ZCTA5 00603,00603
3,6591.0,3721.0,15019.0,134.0,ZCTA5 00606,00606
4,28963.0,14569.0,16707.0,1134.0,ZCTA5 00610,00610


In [7]:
# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

In [8]:
# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Household Income", "Poverty Count", "Poverty Rate",
                      "Unemployment Count", "Unemployment Rate"]]

census_pd.head()

,Zipcode,Population,Household Income,Poverty Count,Poverty Rate,Unemployment Count,Unemployment Rate
0,00601,18450.0,12041.0,10816.0,58.623306,1464.0,7.934959
1,00602,41302.0,15663.0,22409.0,54.256452,4767.0,11.541814
2,00603,53683.0,15485.0,26220.0,48.842278,4341.0,8.086359
3,00606,6591.0,15019.0,3721.0,56.455773,134.0,2.033075
4,00610,28963.0,16707.0,14569.0,50.302110,1134.0,3.915340


In [9]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_school_shootings.csv", encoding="utf-8", index=False)